In [1]:
import sys, os
import numpy as np
import pandas as pd
from utils.utils import read_txt, read_txt_np_int

# INPUT

In [2]:
inputfilename = './inputs/day9A.txt'

inputdata = read_txt(inputfilename)

In [17]:
compressed_files = inputdata[0]
len(compressed_files)

19999

## PART 1

In [ ]:
def decompress_files(filedescription):
    decompressed = []
    for i in range(0, len(filedescription), 2):
        fileid = int(i/2)
        fileblocks = int(filedescription[i])
        decompressed += [str(fileid) for repeat in range(fileblocks)]
        try:
            emptyblocks = int(filedescription[i+1])
            decompressed += ['.' for repeat in range(emptyblocks)]
        except IndexError:
            pass
    return decompressed

def move_blocks(filedescription):
    empty_positions = list(np.where(filedescription == '.')[0])
    filled_positions = list(np.where(filedescription != '.')[0])

    empty_index = empty_positions.pop(0)
    block_index = filled_positions.pop(-1)
    return empty_positions, filled_positions
    


In [35]:
uncompressed_files = np.array(decompress_files(compressed_files))
a, b =move_blocks(uncompressed_files)
a

[np.int64(2),
 np.int64(3),
 np.int64(4),
 np.int64(12),
 np.int64(13),
 np.int64(14),
 np.int64(15),
 np.int64(16),
 np.int64(17),
 np.int64(18),
 np.int64(19),
 np.int64(29),
 np.int64(30),
 np.int64(31),
 np.int64(32),
 np.int64(33),
 np.int64(34),
 np.int64(35),
 np.int64(36),
 np.int64(41),
 np.int64(42),
 np.int64(43),
 np.int64(44),
 np.int64(45),
 np.int64(55),
 np.int64(56),
 np.int64(58),
 np.int64(59),
 np.int64(60),
 np.int64(61),
 np.int64(62),
 np.int64(75),
 np.int64(76),
 np.int64(77),
 np.int64(78),
 np.int64(79),
 np.int64(80),
 np.int64(81),
 np.int64(82),
 np.int64(83),
 np.int64(91),
 np.int64(96),
 np.int64(97),
 np.int64(98),
 np.int64(99),
 np.int64(104),
 np.int64(105),
 np.int64(106),
 np.int64(107),
 np.int64(108),
 np.int64(109),
 np.int64(116),
 np.int64(127),
 np.int64(128),
 np.int64(129),
 np.int64(130),
 np.int64(131),
 np.int64(139),
 np.int64(140),
 np.int64(141),
 np.int64(142),
 np.int64(143),
 np.int64(144),
 np.int64(148),
 np.int64(149),
 np.int6

## PART 2

In [61]:
def find_resonance_points_2(antenna1, antenna2, map_size):
    resonance_points = set()
    deviation = antenna2 - antenna1
    resonance_point = antenna1
    while resonance_point[0] >= 0 and resonance_point[0] < map_size[0] and resonance_point[1] >= 0 and resonance_point[1] < map_size[1]:
        resonance_points.add(tuple(resonance_point))
        resonance_point = resonance_point - deviation
    resonance_point = antenna2
    while resonance_point[0] >= 0 and resonance_point[0] < map_size[0] and resonance_point[1] >= 0 and resonance_point[1] < map_size[1]:
        resonance_points.add(tuple(resonance_point))
        resonance_point = resonance_point + deviation
    return resonance_points

def find_resonance_points_list_2(antenna_location_list, map_size):
    resonance_points = set()
    for i in range(len(antenna_location_list)):
        for j in range(i+1, len(antenna_location_list)):
            resonance_points.update(find_resonance_points_2(antenna_location_list[i], antenna_location_list[j], map_size))
    
    return resonance_points

In [62]:
find_resonance_points_2(np.array([2, 10]), np.array([3,15]), map_size)

{(np.int64(0), np.int64(0)),
 (np.int64(1), np.int64(5)),
 (np.int64(2), np.int64(10)),
 (np.int64(3), np.int64(15)),
 (np.int64(4), np.int64(20)),
 (np.int64(5), np.int64(25)),
 (np.int64(6), np.int64(30)),
 (np.int64(7), np.int64(35)),
 (np.int64(8), np.int64(40)),
 (np.int64(9), np.int64(45))}

In [63]:
# Solve part2
resonance_points = set()
for antenna_char, antenna_locations_list in antenna_locations_dict.items():
    print(f"Processing antenna {antenna_char}")
    resonance_points.update(find_resonance_points_list_2(antenna_locations_list, map_size))
print(f"Found {len(resonance_points)} resonance points")

Processing antenna 4
Processing antenna 6
Processing antenna c
Processing antenna n
Processing antenna R
Processing antenna i
Processing antenna y
Processing antenna 3
Processing antenna Y
Processing antenna 8
Processing antenna E
Processing antenna x
Processing antenna G
Processing antenna h
Processing antenna 7
Processing antenna 5
Processing antenna e
Processing antenna w
Processing antenna 1
Processing antenna g
Processing antenna v
Processing antenna A
Processing antenna d
Processing antenna 0
Processing antenna D
Processing antenna N
Processing antenna 2
Processing antenna 9
Processing antenna H
Processing antenna r
Processing antenna a
Processing antenna C
Processing antenna q
Processing antenna m
Processing antenna V
Processing antenna M
Processing antenna K
Processing antenna b
Processing antenna I
Processing antenna k
Processing antenna Q
Processing antenna W
Processing antenna X
Processing antenna B
Found 1005 resonance points
